Planificación usando Pyomo - Parte 3
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/workshop-asocio-2018/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/workshop-asocio-2018/tree/master/) para explorar el repositorio usando `nbviewer`. 

### Definición del problema

Se desea realizar el planeamiento de la operacion de un sistema hidrotermico conformado por dos plantas hidraulica y dos termicas. La informacion es la siguiente:

 
 
* **Planificacion:**    4 etapas. 


* **Planta hidraulica 1:**    
    * Vol. maximo (V*) = 100.
    * Caudal max. turbinado (Q*) = 50.
    * Factor conversion ($\rho$) = 1.
    * Aporte por etapa ($A_i$) =  {21, 15, 12, 18}.
    * Volumen inicial ($V_o$) = 75.  
    
    
* **Planta hidraulica 2:**    
    * Vol. maximo (V*) = 100.
    * Caudal max. turbinado (Q*) = 45.
    * Factor conversion ($\rho$) = 1.
    * Aporte por etapa ($A_i$)  = {14, 8, 5, 11}.
    * Volumen inicial ($V_o$) = 75.  
    
    
* **Planta termica 1 :**    
    * Generacion maxima (G*) = 45
    * Costo combustible (CC) = 15. 
    
    
* **Planta termica 2 :**    
    * Generacion maxima (G*) = 40
    * Costo combustible (CC) = 15. 
 
 
* **Racionamiento:**   
    * Costo racionamiento (CR) = 1000 para todas las etapas. 
 
 
* **Demanda:**  80 para todas las etapas 
 

### Inicialización de Pyomo

In [1]:
%%writefile model.py


from pyomo.environ import *

## Creación del modelo abstracto
model = AbstractModel()


Overwriting model.py


### Formulación del modelo abstracto

In [2]:
%%writefile -a model.py

## Datos

# Numero de etapas
model.P = Set(ordered=True)                      
                     
# Numero de plantas hidro
model.HD = Set(ordered=True)                     

# Numero de plantas termicas 
model.TM = Set(ordered=True)                     

# Costo de Racionamiento indexado por etapa
model.CR = Param(model.P)                         

# Costo de Combusstible indexado por termica
model.CC = Param(model.TM)                       

# Volumen inicial del embalse indexado por hidro
model.Vo = Param(model.HD)                         

# Demanda indexada etapa
model.D = Param(model.P)                         
         
# Aportes Hidrologicos indexado por plantas hidro
model.A = Param(model.HD,model.P)                

# Caudal turbinado indexado por plantas hidro
model.QH = Param(model.HD)                       

# Generacion termica maxima indexado por termicas
model.GM = Param(model.TM)                       


Appending to model.py


In [3]:
%%writefile -a model.py

# La energia racionada indexada por etapa
model.R = Var(model.P, within=NonNegativeReals)    

# La generacion termica varia por etapa y por planta
model.G = Var(model.P, model.TM, within=NonNegativeReals)   

# El caudar turbinado varia por etapa y por plantas hidroelectricas
model.Q = Var(model.P, model.HD, within=NonNegativeReals)         

# El volumen final varia por etapa y por la planta hidro en que se encuentre
model.V = Var(model.P, model.HD, within=NonNegativeReals)

# El volumen vertido varia por etapa y por planta hidro
model.S = Var(model.P, model.HD, within=NonNegativeReals)


Appending to model.py


### Funcion objetivo

La funcion objetivo busca minimizar el costo total del racionamiento mas el costo total de la generacion termica:


$$\text{minimize} ~ \sum_{p=1}^{P} ~\Bigg\{ CR * DEF_p + \sum_{t=1}^T CC_t * GT_{p,t} ~\Bigg\}$$







In [4]:
%%writefile -a model.py

## Definicion de la funcion objetivo

def obj_rule(model):
    return (sum(model.CR[i]*model.R[i] for i in model.P) + 
           sum(model.CC[j]*model.G[i,j] for j in model.TM for i in model.P))

model.obj = Objective(rule = obj_rule)


Appending to model.py


### Demanda

Por cada periodo de tiempo $p$ debe cumplirse que:

$$ DEF_p + \Bigg\{ \sum_{h=1}^H \rho_h * Q_{hp} \Bigg\} + \Bigg\{ \sum_{t=1}^T GT_{t p} \Bigg\} = dem_p$$


In [5]:
%%writefile -a model.py

def demanda_rule(model,p):
        return (model.R[p] + 
                sum(model.G[p,j] for j in model.TM) + 
                sum(model.Q[p,j] for j in model.HD) == model.D[p])

model.demanda = Constraint(model.P,rule=demanda_rule)


Appending to model.py


### Continuidad de los volumenes de los embalses

Para el primer periodo, el embalse inicial $Vol_{h,0}$ es conocido, entonces:

$$Vol_{h,1} +Q_{h,1}+Ver_{h,1}=A_{h,1}+ Vol_{h,0}$$

In [6]:
%%writefile -a model.py

def continuidad_rule(model, p, hd):
        if model.P[p] == 1 :
            return model.V[p,hd] + model.Q[p,hd] + model.S[p,hd] == model.Vo[hd]+ model.A[hd,1]     
        else:
            return -model.V[p-1,hd] + model.V[p,hd] + model.Q[p,hd] + model.S[p,hd] ==  model.A[hd,p]    

model.continuidad = Constraint(model.P, model.HD, rule=continuidad_rule)


Appending to model.py


### Caudal turbinado maximo

In [7]:
%%writefile -a model.py

def qmax_rule(model, p, HD):
    
    return model.Q[p, HD] <= model.QH[HD]

model.qmax = Constraint(model.P, model.HD, rule = qmax_rule)


Appending to model.py


### Generacion térmica maxima

In [8]:
%%writefile -a model.py

def gtmax_rule(model,p,TM):
    return model.G[p,TM] <= model.GM[TM]

model.gtmax = Constraint(model.P, model.TM, rule=gtmax_rule)


Appending to model.py


### Datos particulares del caso

In [9]:
%%writefile model.dat

set P  := 1 2 3 4 ;
set HD := 1 2 ;
set TM := 1 2 ;

param CR := 
1 1000
2 1000
3 1000
4 1000 ;

param CC := 
1 15
2 15 ;

param Vo :=
1 75
2 75 ;

param D :=
1 80
2 80
3 80
4 80 ;

param A :=
1 1 21
2 1 14
1 2 15
2 2 8
1 3 12
2 3 5
1 4 18
2 4 11 ;

param QH := 
1 50 
2 45 ;

param GM := 
1 45
2 40 ;

Writing model.dat


# Solucion
[Contenido](#Contenido)

In [10]:
!pyomo solve --solver=glpk model.py model.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.04] Applying solver
[    0.06] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 990.0
    Solver results file: results.yml
[    0.06] Applying Pyomo postprocessing actions
[    0.07] Pyomo Finished


In [11]:
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 990.0
  Upper bound: 990.0
  Number of objectives: 1
  Number of constraints: 29
  Number of variables: 37
  Number of nonzeros: 67
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.008777141571044922
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

In [12]:
!rm model.py model.dat

Planificación usando Pyomo - Parte 3
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/workshop-asocio-2018/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/workshop-asocio-2018/tree/master/) para explorar el repositorio usando `nbviewer`. 